# Exercise 3
Ole Gunnar Hovland and Alexander Hatle

In [1]:
%matplotlib inline
import numpy as np
import time
import matplotlib.pyplot as plt
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
from matplotlib import cm
plt.rcParams.update(newparams)

In [12]:
def tridiag(v, d, w, N):
    # Help function 
    # Returns a tridiagonal matrix A=tridiag(v, d, w) of dimension N x N.
    if len(v) + len(d) + len(w) > 3:
        A = np.diag(v, -1) + np.diag(d) + np.diag(w, 1)
    else:
        e = np.ones(N)        # array [1,1,...,1] of length N
        A = v*np.diag(e[1:],-1) + d * np.diag(e) + w * np.diag(e[1:],1)
    return A

In [38]:
def crank(g, params, BC, M=10, N=100, R = 1, T=0.5):
    # Input: 
    #       g: initial function for t=0
    #       M, N: number of grid intervals in the x- and t directions
    #       T: end of integration
    # Output: 
    #       x, t: the gridpoints in the x- and t- directions 
    #       U: An array with the numerical solution.

    

    # Set the stepsizes
    h = R/M     # Stepsize in space
    k = T/N     # Stepsize in time

    # Parameters
    sigma = params[0]
    r = params[1]
    c = params[2]
    K = params[3]

    # Helping parameters
    alpha = sigma**2 * k / (4 * h**2)
    beta = r * k / (4*h)
    gamma = c * k / 2

    # Print the stepsizes, and r=k/h^2.
    print('h={:.4f}, k={:.4f}'.format(h,k))

    U = np.zeros((M+1,N+1))    # Array to store the solution, boundaries included.
    x = np.linspace(0,R,M+1)   # Gridpoints on the x-axis
    t = np.linspace(0,T,N+1)   # Gridpoints on the t-axis
    U[:,0] = g(x, K)              # Initial values, endpoints included
    
    # Constructing A
    lowdiag = -alpha * x[2:-1]**2 - beta * x[2:-1]         # M-2
    diag =  1 + alpha * x[1:-1]**2 + gamma                   # M-1
    updiag = -alpha * x[1:-2]**2 + beta * x[1:-2]         # M-2
    print(len(lowdiag), len(diag), len(updiag))
    A = tridiag(lowdiag, diag, updiag, M-1)

    

    # Constructing B
    lowdiag = alpha * x[2:-1]**2 + beta * x[2:-1]  
    diag = 1 - alpha * x[1:-1]**2 - gamma 
    updiag = alpha * x[2:-1]**2 + beta * x[2:-1] 
    B = tridiag(lowdiag, diag, updiag, M-1)

    # Constructing p and q
    p = np.zeros(M-1)
    p[-1] = (-alpha * x[-1]**2 - beta * x[-1]) * U[-1, 0]
    q = -p

    # Boundary functions
    if BC == 'EP':
        def bndry(t):
            return K * np.exp(-c * t)
    elif BC == 'binary':
        def bndry(t):
            return 0

    # Main loop 
    for n in range(N):
        tempvec = B.dot(U[1:-1, n]) + q - p
        U[1:-1, n+1] = np.linalg.solve(A, tempvec)  # [1:-1] -> excluding endpoints
        # Boundary
        U[0, n+1] = bndry(t[n+1])
    return x, t, U

In [42]:

def european_put(x, K):
    sol = np.zeros(x.shape)
    return (K-x)[K-x > 0]
def binary_call(x, K):
    if x >= K:
        return 1
    else:
        return 0

In [43]:
x, t, U = crank(g = european_put, params = [1,1,0.5,0.5], BC = 'EP', T = 0.1)

plt.figure(1)
plt.clf()
tplots = np.linspace(0,t[-1],6)
k = t[1]-t[0]
for tn in tplots:
    n = int(tn/k)
    tn = n*k
    plt.plot(x,U[:,n],'-o',label='t={:.1f}'.format(tn))
plt.xlabel('x')
plt.ylabel('u(x,t)')
plt.legend()
plt.show()

# Make a 3-d plot
plot_solution(x, t, U, txt='Solution')
plt.show()

plt.plot(x, european_put(x, 0.5))



h=0.1000, k=0.0010


ValueError: could not broadcast input array from shape (5) into shape (11)

In [34]:
def plot_solution(x, t, U, txt='Solution'):
    # Plot the solution of the heat equation
    
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    T, X = np.meshgrid(t,x)
    # ax.plot_wireframe(T, X, U)
    ax.plot_surface(T, X, U, cmap=cm.coolwarm)
    ax.view_init(azim=30)              # Rotate the figure
    plt.xlabel('t')
    plt.ylabel('x')
    plt.title(txt);